# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb8c4bf75b0>
├── 'a' --> tensor([[-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741]])
└── 'x' --> <FastTreeValue 0x7fb8c4bf7370>
    └── 'c' --> tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                        [ 0.0246, -0.4271, -0.1351,  0.4276],
                        [-1.7794, -0.5550, -0.1028,  0.2163]])

In [4]:
t.a

tensor([[-0.2425,  0.0157,  0.9027],
        [-0.2347, -0.6570,  2.4741]])

In [5]:
%timeit t.a

64.1 ns ± 0.0705 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb8c4bf75b0>
├── 'a' --> tensor([[-0.7234,  0.5375,  0.6930],
│                   [ 1.1453,  0.0693, -0.6252]])
└── 'x' --> <FastTreeValue 0x7fb8c4bf7370>
    └── 'c' --> tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                        [ 0.0246, -0.4271, -0.1351,  0.4276],
                        [-1.7794, -0.5550, -0.1028,  0.2163]])

In [7]:
%timeit t.a = new_value

60.1 ns ± 0.0274 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2425,  0.0157,  0.9027],
               [-0.2347, -0.6570,  2.4741]]),
    x: Batch(
           c: tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163]]),
       ),
)

In [10]:
b.a

tensor([[-0.2425,  0.0157,  0.9027],
        [-0.2347, -0.6570,  2.4741]])

In [11]:
%timeit b.a

55.1 ns ± 0.0224 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1218,  0.0859,  1.4875],
               [ 1.5022, -0.8010,  1.0697]]),
    x: Batch(
           c: tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.974 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

840 ns ± 0.0684 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 18 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb80ae38d30>
├── 'a' --> tensor([[[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]],
│           
│                   [[-0.2425,  0.0157,  0.9027],
│                    [-0.2347, -0.6570,  2.4741]]])
└── 'x' --> <FastTreeValue 0x7fb80ae386d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 48.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb8c0b69280>
├── 'a' --> tensor([[-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741],
│                   [-0.2425,  0.0157,  0.9027],
│                   [-0.2347, -0.6570,  2.4741]])
└── 'x' --> <FastTreeValue 0x7fb80ae60f40>
    └── 'c' --> tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                        [ 0.0246, -0.4271, -0.1351,  0.4276],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 49.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.6 µs ± 46.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.4680,  0.6508, -0.3377, -0.2800],
                       [ 0.0246, -0.4271, -0.1351,  0.4276],
                       [-1.7794, -0.5550, -0.1028,  0.2163]],
              
                      [[ 0.4680,  0.6508, -0.3377, -0.2800],
                       [ 0.0246, -0.4271, -0.1351,  0.4276],
                       [-1.7794, -0.5550, -0.1028,  0.2163]],
              
                      [[ 0.4680,  0.6508, -0.3377, -0.2800],
                       [ 0.0246, -0.4271, -0.1351,  0.4276],
                       [-1.7794, -0.5550, -0.1028,  0.2163]],
              
                      [[ 0.4680,  0.6508, -0.3377, -0.2800],
                       [ 0.0246, -0.4271, -0.1351,  0.4276],
                       [-1.7794, -0.5550, -0.1028,  0.2163]],
              
                      [[ 0.4680,  0.6508, -0.3377, -0.2800],
                       [ 0.0246, -0.4271, -0.1351,  0.4276],
                       [-1.7794, -0.5550, -0.1028,  0.2163]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163],
                      [ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163],
                      [ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163],
                      [ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163],
                      [ 0.4680,  0.6508, -0.3377, -0.2800],
                      [ 0.0246, -0.4271, -0.1351,  0.4276],
                      [-1.7794, -0.5550, -0.1028,  0.2163],
                      [ 0.4680,  0.6508, -0.3377, -0.2800],
                   

In [28]:
%timeit Batch.cat(batches)

161 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 538 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
